<span style='color:#009999'> <span style='font-family:serif'> <font size="15"> **CMIP6** - Coupled Model Intercomparison Project Phase 6

<span style='color:#ff6666'><font size="5">**Additional Requirements**

- <font size="3"><span style='color:Black'> None.


 <span style='color:#ff6666'><font size="5"> **Objectives**
- <font size="3"><span style='color:Black'> To demonstrate remote access via tokens to ESGF Portal.
- <font size="3"><span style='color:Black'> To access and subset remote data implementing the DAP2 Protocol.
- <font size="3"><span style='color:Black'> Understand the subtle differences between DAP2 and DAP4.
- <font size="3"><span style='color:Black'> To identify when an OPeNDAP server only implements DAP2.



<span style='color:#ff6666'><font size="5"> **Browsing Data:**

The <font size="3.5"><span style='color:#0066cc'>**Earth System Grid Federation** <font size="3.5"><span style='color:black'> [ESGF](https://aims2.llnl.gov/search/cmip6/) Contains a broad of model output (e.g, CMIP3, CMIP5, [CMIP6](https://pcmdi.llnl.gov/CMIP6/), E3SM) from which you can obtain OPeNDAP URLs for data variables. TO access the ESGF Node and browse data [click here](https://aims2.llnl.gov/search/cmip6/).

<img src="img/ESGF.png" alt="drawing" width="750"/>    



In [6]:
import matplotlib.pyplot as plt
import numpy as np
from pydap.client import open_url
import cartopy.crs as ccrs

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **CMIP6 Access via OPeNDAP**

 <font size="3.5">You can also directly inspect a THREDDS catalog for [CMIP6](https://crd-esgf-drc.ec.gc.ca/thredds/catalog/esgB_dataroot/AR6/CMIP6/catalog.html). For example, you can navigate to `CDRMIP/CCCma/CanESM5/esm-pi-cdr-pulse/r2i1p2f1/Eday/ts/gn/v20190429` and access [ts data](https://crd-esgf-drc.ec.gc.ca/thredds/dodsC/esgB_dataroot/AR6/CMIP6/CDRMIP/CCCma/CanESM5/esm-pi-cdr-pulse/r2i1p2f1/Eday/ts/gn/v20190429/ts_Eday_CanESM5_esm-pi-cdr-pulse_r2i1p2f1_gn_54510101-56501231.nc.html) via OPeNDAP DAP2 protocol.



In [7]:
url = "https://crd-esgf-drc.ec.gc.ca/thredds/dodsC/esgB_dataroot/AR6/CMIP6/CDRMIP/CCCma/CanESM5/esm-pi-cdr-pulse/r2i1p2f1/Eday/ts/gn/v20190429/ts_Eday_CanESM5_esm-pi-cdr-pulse_r2i1p2f1_gn_54510101-56501231.nc"


<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Create dataset access via pydap**

- By default protocol='dap2', however this behavior may change in the nearby future.


In [10]:
%%time
ds = open_url(url, protocol='dap2')

CPU times: user 65 ms, sys: 8.59 ms, total: 73.6 ms
Wall time: 1.17 s


In [11]:
ds.tree()

.esgB_dataroot/AR6/CMIP6/CDRMIP/CCCma/CanESM5/esm-pi-cdr-pulse/r2i1p2f1/Eday/ts/gn/v20190429/ts_Eday_CanESM5_esm-pi-cdr-pulse_r2i1p2f1_gn_54510101-56501231.nc
├──time
├──time_bnds
├──lat
├──lat_bnds
├──lon
├──lon_bnds
└──ts
   ├──ts
   ├──time
   ├──lat
   └──lon


In [13]:
ts = ds['ts']

In [15]:
ts

<GridType with array 'ts' and maps 'time', 'lat', 'lon'>

In [18]:
ts.attributes

{'standard_name': 'surface_temperature',
 'long_name': 'Surface Temperature',
 'comment': 'Temperature of the lower boundary of the atmosphere',
 'units': 'K',
 'original_name': 'GT',
 'cell_methods': 'area: time: mean',
 'cell_measures': 'area: areacella',
 'history': '2019-08-20T21:03:55Z altered by CMOR: Reordered dimensions, original order: lat lon time. 2019-08-20T21:03:55Z altered by CMOR: replaced missing value flag (1e+38) and corresponding data with standard missing value (1e+20).',
 'missing_value': 1e+20,
 '_FillValue': 1e+20,
 '_ChunkSizes': [1, 64, 128]}

In [19]:
ts.tree()

.ts
├──ts
├──time
├──lat
└──lon


In [22]:
ts.shape

(73000, 64, 128)